In [195]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [196]:
train = pd.read_csv('/home/necati/machine_learning/Kaggle/Kaggle_datasets/trainSpaceship.csv')
test = pd.read_csv('/home/necati/machine_learning/Kaggle/Kaggle_datasets/testSpaceship.csv')

In [197]:
train.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [198]:
train.drop(columns=['PassengerId', 'Name', 'Cabin'], inplace=True)
Id_column = test['PassengerId']
test.drop(columns=['PassengerId', 'Name', 'Cabin'], inplace=True)

In [199]:
print(train.isnull().sum())
print('\nThe shape of the training data is', train.shape)
print(test.isnull().sum())
print(test.shape)

HomePlanet      201
CryoSleep       217
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64

The shape of the training data is (8693, 11)
HomePlanet       87
CryoSleep        93
Destination      92
Age              91
VIP              93
RoomService      82
FoodCourt       106
ShoppingMall     98
Spa             101
VRDeck           80
dtype: int64
(4277, 10)


In [200]:
train.fillna(train.median(numeric_only=True), inplace=True)
for col in train.select_dtypes(include='object').columns:
    train.fillna({col:train[col].mode()[0]}, inplace=True)
test.fillna(test.median(numeric_only=True), inplace=True)
for col in test.select_dtypes(include=['object']).columns:
    test.fillna({col: test[col].mode()[0]}, inplace=True)

/tmp/ipykernel_5440/2694146047.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train.fillna({col:train[col].mode()[0]}, inplace=True)
/tmp/ipykernel_5440/2694146047.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test.fillna({col: test[col].mode()[0]}, inplace=True)


In [201]:
scaler = StandardScaler()
numeric_columns = train.select_dtypes(include=['number']).columns
train[numeric_columns] = scaler.fit_transform(train[numeric_columns])
test[numeric_columns] = scaler.transform(test[numeric_columns])
train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,TRAPPIST-1e,0.711945,False,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,False
1,Earth,False,TRAPPIST-1e,-0.334037,False,-0.168073,-0.275387,-0.241771,0.217158,-0.224205,True
2,Europa,False,TRAPPIST-1e,2.036857,True,-0.268001,1.959998,-0.283579,5.695623,-0.219796,False
3,Europa,False,TRAPPIST-1e,0.293552,False,-0.333105,0.523010,0.336851,2.687176,-0.092818,False
4,Earth,False,TRAPPIST-1e,-0.891895,False,0.125652,-0.237159,-0.031059,0.231374,-0.261240,True


In [202]:
train_wo_target = train.drop(columns=['Transported'])
train_wo_target = pd.get_dummies(train_wo_target, columns=train_wo_target.select_dtypes(include=['object']).columns)
bool_columns = train_wo_target.select_dtypes(include=[bool]).columns
train_wo_target[bool_columns] = train_wo_target[bool_columns].astype(int)
train_wo_target.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,0,0.711945,0,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,0,1,0,0,0,1
1,0,-0.334037,0,-0.168073,-0.275387,-0.241771,0.217158,-0.224205,1,0,0,0,0,1
2,0,2.036857,1,-0.268001,1.959998,-0.283579,5.695623,-0.219796,0,1,0,0,0,1
3,0,0.293552,0,-0.333105,0.523010,0.336851,2.687176,-0.092818,0,1,0,0,0,1
4,0,-0.891895,0,0.125652,-0.237159,-0.031059,0.231374,-0.261240,1,0,0,0,0,1


In [203]:
train = pd.concat([train_wo_target, train[train.columns[-1]]], axis=1)
train['Transported'] = train['Transported'].astype(int)
train.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported
0,0,0.711945,0,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,0,1,0,0,0,1,0
1,0,-0.334037,0,-0.168073,-0.275387,-0.241771,0.217158,-0.224205,1,0,0,0,0,1,1
2,0,2.036857,1,-0.268001,1.959998,-0.283579,5.695623,-0.219796,0,1,0,0,0,1,0
3,0,0.293552,0,-0.333105,0.523010,0.336851,2.687176,-0.092818,0,1,0,0,0,1,0
4,0,-0.891895,0,0.125652,-0.237159,-0.031059,0.231374,-0.261240,1,0,0,0,0,1,1


In [204]:
train_set, val_set = train_test_split(train, test_size=0.2, random_state=13)

X_train = train_set[train_set.columns[:-1]]
y_train = train_set[train_set.columns[-1]]

X_val = val_set[val_set.columns[:-1]]
y_val = val_set[val_set.columns[-1]]


In [205]:
model1 = RandomForestClassifier(random_state=13)
model1.fit(X_train, y_train)

RandomForestClassifier(random_state=13)

In [206]:
train_pred1 = model1.predict(X_train)
val_pred1 = model1.predict(X_val)

In [207]:
print(accuracy_score(y_train, train_pred1))
print(accuracy_score(y_val, val_pred1))

0.9314063848144952
0.7855089131684876


In [208]:
test = pd.get_dummies(test, columns= test.select_dtypes(include='object').columns)
test.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,True,-0.124841,False,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,True,False,False,False,False,True
1,False,-0.682698,False,-0.333105,-0.275387,-0.283579,2.237598,-0.263003,True,False,False,False,False,True
2,True,0.154088,False,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,False,True,False,True,False,False
3,False,0.642213,False,-0.333105,3.887680,-0.283579,-0.109808,0.252842,False,True,False,False,False,True
4,False,-0.612966,False,-0.317964,-0.281027,0.778343,-0.270626,-0.263003,True,False,False,False,False,True


In [209]:
bool_columns_test = test.select_dtypes(include=[bool]).columns
test[bool_columns_test] = test[bool_columns_test].astype(int)
test.head()

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1,-0.124841,0,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,1,0,0,0,0,1
1,0,-0.682698,0,-0.333105,-0.275387,-0.283579,2.237598,-0.263003,1,0,0,0,0,1
2,1,0.154088,0,-0.333105,-0.281027,-0.283579,-0.270626,-0.263003,0,1,0,1,0,0
3,0,0.642213,0,-0.333105,3.887680,-0.283579,-0.109808,0.252842,0,1,0,0,0,1
4,0,-0.612966,0,-0.317964,-0.281027,0.778343,-0.270626,-0.263003,1,0,0,0,0,1


In [210]:
test_pred1 = model1.predict(test)

In [213]:
submission1 = pd.DataFrame({'PassengerId': Id_column,
                            'Transported': test_pred1.astype(bool)})
submission1.to_csv('Spaceship_submission1.csv', index=False)

In [212]:
test_pred1.astype(bool)

array([ True, False,  True, ...,  True,  True, False])